In [7]:
# -*- coding: utf-8 -*-
"""Self_Driving_CNN_model.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1q9taOjxGJxIKqvotSXIo1tXOu90g7acB
"""

import numpy as np
import pandas as pd
import os
from os.path import join, isfile
from PIL import Image
import torch
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
!pip install openpyxl

# Change directory to the data directory from hw1
#os.chdir(r"C:\Users\blume\Desktop\School\Self Driving Cars\Project\train_images")
count = 0
train_x_im = []
parent_fldr = "D:/CMU/CMU Classes/Self Driving Cars/Project/Perception/Data/train_images"
dir_list = os.listdir(r"D:/CMU/CMU Classes/Self Driving Cars/Project/Perception/Data/train_images")

test_fldr = "D:/CMU/CMU Classes/Self Driving Cars/Project/Perception/Data/test_images"
test_list = os.listdir(r"D:/CMU/CMU Classes/Self Driving Cars/Project/Perception/Data/test_images")

# Get labels for all images from given excel file
labels = pd.read_excel(r"D:/CMU/CMU Classes/Self Driving Cars/Project/Perception/Data/trainval/labels.xlsx")
train_labels = []
label_count = 0

# Open all the images from the files list for both train and test data and append to a large tensor
# Normalize the data so it is easier to process
for file in dir_list:
    curr_label = labels.iat[label_count,1]
    label_count = label_count + 1
    train_labels.append(curr_label)

    # Process the image to make it standard for the NN
    full_file = parent_fldr + "/" + file
    with Image.open(full_file) as img:
        img = img.resize((300, 300))
        img_data = np.asarray(img)
        img_data = img_data.astype(np.float32) # / 255.0
        img_tensor = torch.from_numpy(img_data).float()
        train_x_im.append(img_tensor)

train_x = torch.stack(train_x_im)
print(np.shape(train_x))
print(np.shape(train_labels))
# Rearange tensor
#train_shuf, labels_shuf = shuffle(train_x, train_labels, random_state=0)


     -------------------------------------- 250.0/250.0 kB 5.1 MB/s eta 0:00:00


MemoryError: Unable to allocate 1.03 MiB for an array with shape (300, 300, 3) and data type float32

In [19]:
test_true_im = []
# Test data tensor
for image in test_list:
    # Process the image to make it standard for the NN
    full_file = test_fldr + "/" + image
    with Image.open(full_file) as img:
        img = img.resize((300, 300))
        img_data = np.asarray(img)
        img_data = img_data.astype(np.float32) # / 255.0
        img_tensor = torch.from_numpy(img_data).float()
        test_true_im.append(img_tensor)

test_true = torch.stack(test_true_im)
#X_train, X_test, y_train, y_test = train_test_split(train_x, train_labels, test_size=0.33, random_state=42)


In [11]:


#X_train = X_train.unsqueeze(0)
#X_test = X_test.unsqueeze(0)
#X_train = X_train.permute(0, 3, 1, 2)
#X_test = X_test.permute(0, 3, 1, 2)
#test_x_im = test_x_im.permute(0,2,1,3)
print(np.shape(test_x_im))
print(np.shape(X_train))
print(np.shape(X_test))

# Create a tensor for the labels as well
y_train = np.array(y_train)
y_train = y_train.astype(int)
y_test = np.array(y_test)
y_test = y_test.astype(int)
train_label = torch.tensor(y_train).long()
test_label = torch.tensor(y_test).long()


torch.Size([5262, 3, 300, 300])
torch.Size([5073, 3, 300, 300])
torch.Size([2500, 3, 300, 300])


In [20]:
test_true = test_true.permute(0,3,1,2)

In [17]:

import torch.nn as nn
# Create CNN model with various layers. Apply pooling and RELU
# Make sure to set the linear layers correctly
class Conv2D(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3,20,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(20,64,5),
            nn.ReLU(),
            nn.MaxPool2d(4,4),
            nn.Conv2d(64,64,7),
            nn.ReLU(),
            nn.MaxPool2d(4,4),
            
            nn.Flatten(),
            #nn.Linear(3136,6000),
            #nn.ReLU(),
            nn.Linear(3136,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,3)
        )
    
    def forward(self, x):
        return self.network(x)

model = Conv2D()

# Create your train and test datasets with labels
batch_size = 200
train_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(X_train,train_label),
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(X_test,test_label),
                                           batch_size=batch_size,
                                           shuffle=True)
test_true_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(test_x_im),
                                           batch_size=batch_size,
                                           shuffle=True)




In [37]:
true_test = np.zeros([2631,])
true_test = np.array(true_test)
true_label = torch.tensor(true_test).long()
test_true_loader = torch.utils.data.DataLoader(dataset=torch.utils.data.TensorDataset(test_true,true_label),
                                           batch_size=batch_size,
                                           shuffle=False)

In [14]:

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 7
train_loss_tot = []
train_acc_tot = []
test_loss_tot = []
test_acc_tot = []
 
# Loop through the number of epochs
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
 
    # set model to train mode
    model.train()
    # iterate over the training data
    for inputs, labels in train_loader:
        #inputs = inputs.reshape(batch_size,1,300,300)
        optimizer.zero_grad()
        outputs = model(inputs)
        #compute the loss
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # increment the running loss and accuracy
        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()
 
    # calculate the average training loss and accuracy
    train_loss /= len(train_loader)
    train_loss_tot.append(train_loss)
    train_acc /= len(train_loader.dataset)
    train_acc_tot.append(train_acc)
 
    # set the model to evaluation mode
    model.eval()
    with torch.no_grad():
        # Validation data
        for inputs, labels in test_loader:
            #inputs = inputs.reshape(batch_size,1,300,300) 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            test_acc += (outputs.argmax(1) == labels).sum().item()

        
 
    # calculate the average validation loss and accuracy
    test_loss /= len(test_loader)
    test_loss_tot.append(test_loss)
    test_acc /= len(test_loader.dataset)
    test_acc_tot.append(test_acc)
    print(f'Epoch {epoch+1}/{num_epochs}, train loss: {train_loss:.4f}, train acc: {train_acc:.4f}, test loss: {test_loss:.4f}, test acc: {test_acc:.4f}')


Epoch 1/7, train loss: 2.0956, train acc: 0.5425, test loss: 0.8050, test acc: 0.6348
Epoch 2/7, train loss: 0.8007, train acc: 0.6318, test loss: 0.8301, test acc: 0.6348
Epoch 3/7, train loss: 0.8049, train acc: 0.6322, test loss: 0.7835, test acc: 0.6348
Epoch 4/7, train loss: 0.7797, train acc: 0.6328, test loss: 0.7976, test acc: 0.6128
Epoch 5/7, train loss: 0.7814, train acc: 0.6320, test loss: 0.7740, test acc: 0.6324
Epoch 6/7, train loss: 0.7629, train acc: 0.6391, test loss: 0.7645, test acc: 0.6324
Epoch 7/7, train loss: 0.7483, train acc: 0.6412, test loss: 0.7564, test acc: 0.6340


In [27]:
# Given test data set 
outputs_true_all = []
with torch.no_grad():
    for inputs, true_label in test_true_loader:
        #inputs = inputs.reshape(batch_size,1,300,300) 
        outputs_true = model(inputs)
        maxes = outputs_true[0].argmax(1)
        outputs_true_all.append(maxes)

all_maxes = np.array(all_maxes)
predictions = outputs_true_all.argmax(1)
d = {'guid/image': test_list, 'label': predictions}
df = pd.DataFrame(data=d)


ValueError: All arrays must be of the same length

In [29]:
predictions.shape
predictions = outputs_true_all.argmax(1)
d = {'guid/image': test_list, 'label': predictions}
df = pd.DataFrame(data=d)

AttributeError: 'list' object has no attribute 'argmax'

In [36]:
outputs_true_all[0].shape
outputs_true_all[0].argmax(1)


array([1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2,
       1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2,
       2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1,
       2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 0,
       2, 1], dtype=int64)

In [46]:
arr = np.array(outputs_true)
maxes = arr.argmax(1)
print(maxes)
all_maxes = []
all_maxes.append(maxes)

[2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]


In [56]:
#all_maxes.append(maxes)
print(all_maxes)
all_maxes = np.array(all_maxes)
print(all_maxes.shape)
new_maxes = np.reshape(all_maxes,(-1,1))
print(new_maxes.shape)

[[2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]
 [2 1 2 1 2 2 1 1 2 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1]]
(9, 31)
(279, 1)
